In [508]:
import pandas as pd
import numpy as np
import torch

In [509]:
wineDF = pd.read_csv('../../data/text/winequality-white.csv', sep=';')
wineDF

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.0              0.27         0.36            20.7      0.045   
1               6.3              0.30         0.34             1.6      0.049   
2               8.1              0.28         0.40             6.9      0.050   
3               7.2              0.23         0.32             8.5      0.058   
4               7.2              0.23         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
4893            6.2              0.21         0.29             1.6      0.039   
4894            6.6              0.32         0.36             8.0      0.047   
4895            6.5              0.24         0.19             1.2      0.041   
4896            5.5              0.29         0.30             1.1      0.022   
4897            6.0              0.21         0.38             0.8      0.020   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                    45.0                 170.0  1.00100  3.00       0.45   
1                    14.0                 132.0  0.99400  3.30       0.49   
2                    30.0                  97.0  0.99510  3.26       0.44   
3                    47.0                 186.0  0.99560  3.19       0.40   
4                    47.0                 186.0  0.99560  3.19       0.40   
...                   ...                   ...      ...   ...        ...   
4893                 24.0                  92.0  0.99114  3.27       0.50   
4894                 57.0                 168.0  0.99490  3.15       0.46   
4895                 30.0                 111.0  0.99254  2.99       0.46   
4896                 20.0                 110.0  0.98869  3.34       0.38   
4897                 22.0                  98.0  0.98941  3.26       0.32   

      alcohol  quality  
0         8.8        6  
1         9.5        6  
2        10.1        6  
3         9.9        6  
4         9.9        6  
...       ...      ...  
4893     11.2        6  
4894      9.6        5  
4895      9.4        6  
4896     12.8        7  
4897     11.8        6  

[4898 rows x 12 columns]

In [510]:
# target을 Long 자료형으로 줘야해서 정규화시킴
wineDF['quality'] =  wineDF['quality'] - 3

In [511]:
wineDF['quality'].unique()

array([3, 2, 4, 5, 1, 0, 6], dtype=int64)

In [512]:
wineDF.shape

(4898, 12)

In [513]:
wineDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [514]:
wineT = torch.as_tensor(wineDF.values, dtype=torch.float32)
wineT

tensor([[ 7.0000,  0.2700,  0.3600,  ...,  0.4500,  8.8000,  3.0000],
        [ 6.3000,  0.3000,  0.3400,  ...,  0.4900,  9.5000,  3.0000],
        [ 8.1000,  0.2800,  0.4000,  ...,  0.4400, 10.1000,  3.0000],
        ...,
        [ 6.5000,  0.2400,  0.1900,  ...,  0.4600,  9.4000,  3.0000],
        [ 5.5000,  0.2900,  0.3000,  ...,  0.3800, 12.8000,  4.0000],
        [ 6.0000,  0.2100,  0.3800,  ...,  0.3200, 11.8000,  3.0000]])

In [515]:
wineT.shape, wineT.ndim

(torch.Size([4898, 12]), 2)

In [516]:
features = wineT[:,:-1]
target = wineT[:,-1]

In [517]:
features.shape, target.shape

(torch.Size([4898, 11]), torch.Size([4898]))

In [518]:
target = target.long()

In [519]:
# 랜덤 시드 설정
torch.manual_seed(30)

In [520]:
# 모델 구현
# cost function으로 사용될 cross_entropy가 자체적으로 softmax를 적용하기에 학습모델에서 따로 사용할 필요가 없음
model = torch.nn.Linear(11, 7)

In [521]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [522]:
for p in model.parameters():
    print(p)

Parameter containing:
tensor([[ 0.2417,  0.1486, -0.0171,  0.2254,  0.1449,  0.1713,  0.2379,  0.0747,
         -0.0437,  0.2063,  0.1480],
        [ 0.0712,  0.2341, -0.0526, -0.1961, -0.0949,  0.2640, -0.2766, -0.2336,
          0.1189,  0.2267, -0.1108],
        [-0.1213,  0.1704,  0.0923, -0.0470, -0.1116,  0.0491, -0.2619, -0.0930,
          0.2798, -0.1513, -0.2307],
        [ 0.0558,  0.0082,  0.0454, -0.1772, -0.0749, -0.1927, -0.0199, -0.2716,
          0.0457, -0.2123, -0.0930],
        [-0.1712, -0.1378,  0.2831,  0.1988, -0.2753,  0.0108, -0.1041, -0.0478,
         -0.1587, -0.0895, -0.0823],
        [-0.0279,  0.1905,  0.1920,  0.0013, -0.2177,  0.1128, -0.1481,  0.2706,
          0.0140,  0.2640, -0.2142],
        [ 0.2587, -0.2308,  0.2140,  0.0808, -0.0006,  0.0095,  0.1942, -0.1357,
         -0.0409,  0.2274,  0.2594]], requires_grad=True)
Parameter containing:
tensor([0.1560, 0.0845, 0.2895, 0.1067, 0.1506, 0.1057, 0.1110],
       requires_grad=True)


In [523]:
def training():
    nb_epochs = 1000
    for epoch in range(nb_epochs + 1):

        # H(x) 계산 => 예측값 계산
        hypothesis = model(features)

        # cost 계산 => cross_entropy
        cost = torch.nn.functional.cross_entropy(hypothesis, target)

        # cost로 W,b값 업데이트
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # 20번마다 로그 출력
        if epoch % 10 == 0:
            # argmax 메소드를 이용해 정확도 산출
            accuracy = sum(hypothesis.argmax(dim=1) == target) / len(hypothesis)
            print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f},'
                  f' Accuracy: {accuracy:.6f}%')

In [524]:
# 시드값에 따라서 정확도가 다른데 41% 정도가 한계인거 같다.
training()

Epoch    0/1000 Cost: 64.783073, Accuracy: 0.004083%
Epoch   10/1000 Cost: 43.030266, Accuracy: 0.002450%
Epoch   20/1000 Cost: 26.007133, Accuracy: 0.002450%
Epoch   30/1000 Cost: 15.365667, Accuracy: 0.241935%
Epoch   40/1000 Cost: 10.519955, Accuracy: 0.262964%
Epoch   50/1000 Cost: 5.909268, Accuracy: 0.258269%
Epoch   60/1000 Cost: 3.829982, Accuracy: 0.299510%
Epoch   70/1000 Cost: 3.639787, Accuracy: 0.327481%
Epoch   80/1000 Cost: 3.527475, Accuracy: 0.335035%
Epoch   90/1000 Cost: 3.429220, Accuracy: 0.337893%
Epoch  100/1000 Cost: 3.335911, Accuracy: 0.340343%
Epoch  110/1000 Cost: 3.244957, Accuracy: 0.341976%
Epoch  120/1000 Cost: 3.155426, Accuracy: 0.342997%
Epoch  130/1000 Cost: 3.066949, Accuracy: 0.343610%
Epoch  140/1000 Cost: 2.979397, Accuracy: 0.344835%
Epoch  150/1000 Cost: 2.892785, Accuracy: 0.346060%
Epoch  160/1000 Cost: 2.807273, Accuracy: 0.348714%
Epoch  170/1000 Cost: 2.723266, Accuracy: 0.349326%
Epoch  180/1000 Cost: 2.641651, Accuracy: 0.349735%
Epoch  